In [61]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [62]:
import os
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface.embeddings import HuggingFaceEndpointEmbeddings
from langchain.schema import Document
import uuid
import re
import pandas as pd

from langchain_qdrant import QdrantVectorStore

from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams

from dotenv import load_dotenv
load_dotenv()


True

In [25]:
files = os.listdir("../docs")
files

['신용정보의 이용 및 보호에 관한 법률(법률)(제20304호)(20240814).pdf',
 '사방사업법(법률)(제19117호)(20230628).pdf',
 '전자서명법 시행규칙(과학기술정보통신부령)(제00056호)(20201210).pdf',
 '금융회사의 지배구조에 관한 법률(법률)(제19913호)(20240703).pdf',
 '대부업 등의 등록 및 금융이용자 보호에 관한 법률(법률)(제20714호)(20250722).pdf',
 '금융소비자 보호에 관한 법률 시행령(대통령령)(제34888호)(20240915).pdf',
 '법원 행정전자서명 인증업무에 관한 규칙(대법원규칙)(제02865호)(20191106).pdf',
 '서민의 금융생활 지원에 관한 법률(법률)(제20823호)(20250919).pdf',
 '개인정보 보호위원회 직제(대통령령)(제35331호)(20250225).pdf',
 '온라인투자연계금융업 및 이용자 보호에 관한 법률(법률)(제19700호)(20230914).pdf',
 '여신전문금융업법 시행규칙(총리령)(제01635호)(20200805).pdf',
 '원자력시설 등의 방호 및 방사능 방재 대책법 시행규칙(총리령)(제02000호)(20241224).pdf',
 '여신전문금융업법(법률)(제20716호)(20250422).pdf',
 '금융위원회와 그 소속기관 직제(대통령령)(제35517호)(20250520).pdf',
 '소비자기본법 시행령(대통령령)(제35093호)(20250101).pdf',
 '.DS_Store',
 '개인정보 보호법(법률)(제19234호)(20250313).pdf',
 '검찰사건사무규칙(법무부령)(제01098호)(20250604).pdf',
 '고위공직자범죄수사처의 개인정보 처리에 관한 규정(대통령령)(제30832호)(20200715).pdf',
 '금융실명거래 및 비밀보장에 관한 법률 시행규칙(총리령)(제01406호)(20170726).pdf',
 '전자상ᄀ

In [95]:
FILE_PATH = "docs/"+files[0]
file_id = uuid.uuid4().hex
# PyMuPDF 로더 인스턴스 생성
loader = PyMuPDFLoader(FILE_PATH)

# 문서 로드
docs = loader.load()

In [72]:
def clean_text(text):
    text = text.replace("&lt", "").replace("&amp", "&").replace("&quot", '"').replace("&apos", "'").replace("&nbsp", " ").replace("&gt;", ">").replace("&gt", ">").replace("&lt;", "<").replace("&lt", "<").replace("&#39;", "'").replace("&#39", "'").replace("&#34;", '"').replace("&#34", '"').replace("&gt;", ">").replace("&gt", ">").replace("&lt;", "<").replace("&lt", "<").replace("&nbsp;", " ").replace("&nbsp", " ").replace("&amp;", "&").replace("&amp", "&").replace("&quot;", '"').replace("&quot", '"').replace("&apos;", "'").replace("&apos", "'").replace("&#39;", "'").replace("&#39", "'").replace("&#34;", '"').replace("&#34", '"')
    text = re.sub(r"\s{2,}", " ", text)
    # text = re.sub(r".{2,}", ".", text)
    new_text = []
    for t in text.split("\n"):
        if "삭제" not in t:
            new_text.append(t)
    new_text = "\n".join(new_text)
    PAIR_PATTERN = re.compile(r"<[^<>]*>|\[[^\[\]]*\]")
    PAIR_PATTERN.sub("", new_text)
    return PAIR_PATTERN.sub("", new_text)

In [64]:
chunk_size=512
    
# 법률문서 구분자 (우선순위 순)
legal_separators = [
    "\n제\d+조",      # 조문
    "\n제\d+장",      # 장
    "\n제\d+절",      # 절
    "\n①",           # 항
    "\n②", "\n③", "\n④", "\n⑤", "\n⑥", "\n⑦", "\n⑧", "\n⑨", "\n⑩",
    "\n1.", "\n2.", "\n3.", "\n4.", "\n5.",  # 숫자 항목
    "\n가.", "\n나.", "\n다.", "\n라.", "\n마.",  # 한글 항목
    "\n\n",          # 단락
    "\n",            # 줄바꿈
    " ",             # 공백
]

base_splitter = RecursiveCharacterTextSplitter(
    separators=legal_separators,
    chunk_size=chunk_size,
    length_function=len,
)

In [65]:
def extract_legal_structure(text):
    """텍스트에서 법률 구조 정보 추출"""
    structure = {}
    
    # 조문 찾기
    article_match = re.search(r'제(\d+)조', text)
    if article_match:
        structure['article'] = f"제{article_match.group(1)}조"
    
    # 장 찾기
    chapter_match = re.search(r'제(\d+)장', text)
    if chapter_match:
        structure['chapter'] = f"제{chapter_match.group(1)}장"
    
    # 절 찾기
    section_match = re.search(r'제(\d+)절', text)
    if section_match:
        structure['section'] = f"제{section_match.group(1)}절"
    
    # 항 찾기
    paragraph_matches = re.findall(r'[①②③④⑤⑥⑦⑧⑨⑩]', text)
    if paragraph_matches:
        structure['paragraph'] = paragraph_matches
    
    return structure

In [66]:
def _merge_short_chunks(splits, page_num, min_length=200):
    """500글자 이하의 짧은 청크들을 합치기"""
    if not splits:
        return []
    
    merged_splits = []
    current_chunk = None
    
    for split in splits:
        if current_chunk is None:
            current_chunk = split
        else:
            if len(current_chunk.page_content) < min_length:
                # 내용 합치기
                current_chunk.page_content += "\n\n" + split.page_content
                
                # 메타데이터 업데이트 (법률 구조 정보는 첫 번째 청크 기준)
                current_chunk.metadata.update({
                    # 'merged_chunks': current_chunk.metadata.get('merged_chunks', 1) + 1,
                    'chunk_length': len(current_chunk.page_content),
                    # 'is_merged': True
                })
                
                # 추가 법률 구조 정보가 있으면 업데이트
                if split.metadata.get('legal_article') and not current_chunk.metadata.get('legal_article'):
                    current_chunk.metadata['legal_article'] = split.metadata.get('legal_article')
                if split.metadata.get('legal_chapter') and not current_chunk.metadata.get('legal_chapter'):
                    current_chunk.metadata['legal_chapter'] = split.metadata.get('legal_chapter')
                if split.metadata.get('legal_section') and not current_chunk.metadata.get('legal_section'):
                    current_chunk.metadata['legal_section'] = split.metadata.get('legal_section')
                
                # # 항 정보는 누적
                # current_paragraphs = current_chunk.metadata.get('legal_paragraph', [])
                # new_paragraphs = split.metadata.get('legal_paragraph', [])
                # if isinstance(current_paragraphs, list) and isinstance(new_paragraphs, list):
                #     current_chunk.metadata['legal_paragraph'] = list(set(current_paragraphs + new_paragraphs))
                
            else:
                # 현재 청크가 충분히 크면 저장하고 새로운 청크 시작
                current_chunk.metadata['chunk_length'] = len(current_chunk.page_content)
                merged_splits.append(current_chunk)
                current_chunk = split
    
    # 마지막 청크 처리
    if current_chunk is not None:
        current_chunk.metadata['chunk_length'] = len(current_chunk.page_content)
        merged_splits.append(current_chunk)
    
    # 최종 청크 인덱스 재정렬
    for i, split in enumerate(merged_splits):
        # split.metadata['final_chunk_index'] = i
        split.metadata['page_chunk_id'] = f"page_{page_num}_chunk_{i}"
    
    return merged_splits

In [23]:
all_splits = []
error_files = []
for idx, file in enumerate(files):
    if file.endswith(".pdf") ==  False:
        continue
    FILE_PATH = "/Users/hongbikim/Dev/dacon_finance/docs/" + file
    file_id = uuid.uuid4().hex

    # PyMuPDF 로더 인스턴스 생성
    loader = PyMuPDFLoader(FILE_PATH)

    # 문서 로드
    docs = loader.load()

    for doc in docs:
        # 페이지별로 처리
        page_num = doc.metadata.get('page', 0)
        content = clean_text(doc.page_content)
        
        # 조문/장절 정보 추출
        legal_structure = extract_legal_structure(content)
        
        # 텍스트 분할
        splits = base_splitter.split_text(content)
        
        # 임시로 분할된 청크들 생성
        temp_splits = []
        for i, split_content in enumerate(splits):
            chunk_structure = extract_legal_structure(split_content)
            
            split_doc = Document(
                page_content=split_content,
                metadata={
                    # **doc.metadata,
                    # 'chunk_index': i,
                    'file_name': FILE_PATH.split("/")[-1],
                    'file_id': file_id,
                    'page_chunk_id': f"page_{page_num}_chunk_{i}",
                    # 'legal_article': chunk_structure.get('article'),
                    # 'legal_chapter': chunk_structure.get('chapter'),
                    # 'legal_section': chunk_structure.get('section'),
                    # 'legal_paragraph': chunk_structure.get('paragraph'),
                }
            )
            temp_splits.append(split_doc)
        
        # 짧은 청크들 합치기
        merged_splits = _merge_short_chunks(temp_splits, page_num)
        try:
            print(f"[{idx}/{len(files)}]")
            vector_store.add_documents(documents=merged_splits)
        except Exception as e:
            print(f"Error adding documents for file {i}th {file}: {e}")
            error_files.append(file)
            continue
        all_splits.extend(merged_splits)

[0/106]
[0/106]
[0/106]
[0/106]
[0/106]
[0/106]
[0/106]
[0/106]
[0/106]
[0/106]
[0/106]
[0/106]
[0/106]
[0/106]
[0/106]
[0/106]
[0/106]
[0/106]
[0/106]
[0/106]
[0/106]
[0/106]
[0/106]
[0/106]
[0/106]
[0/106]
[0/106]
[0/106]
[0/106]
[0/106]
[0/106]
[0/106]
[0/106]
[0/106]
[0/106]
[0/106]
[0/106]
[0/106]
[0/106]
[0/106]
[0/106]
[1/106]
[1/106]
[1/106]
[1/106]
[1/106]
[1/106]
[1/106]
[1/106]
[2/106]
[2/106]
[2/106]
[2/106]
[2/106]
[2/106]
[3/106]
[3/106]
[3/106]
[3/106]
[3/106]
[3/106]
[3/106]
[3/106]
[3/106]
[3/106]
[3/106]
[3/106]
[3/106]
[3/106]
[3/106]
[3/106]
[3/106]
[3/106]
[3/106]
[3/106]
[3/106]
[3/106]
[4/106]
[4/106]
[4/106]
[4/106]
[4/106]
[4/106]
[4/106]
[4/106]
[4/106]
[4/106]
[4/106]
[4/106]
[4/106]
[4/106]
[4/106]
[4/106]
[4/106]
[4/106]
[4/106]
[4/106]
[4/106]
[4/106]
[4/106]
[4/106]
[4/106]
[5/106]
[5/106]
[5/106]
[5/106]
[5/106]
[5/106]
[5/106]
[5/106]
[5/106]
[5/106]
[5/106]
[5/106]
[5/106]
[5/106]
[5/106]
[5/106]
[5/106]
[5/106]
[5/106]
[5/106]
[5/106]
[5/106]
[5/106]


In [73]:
# 추가만
all_splits = []
error_files = []
for idx, file in enumerate(["docs_none/rfc4408.pdf"]):
    if file.endswith(".pdf") ==  False:
        continue
    FILE_PATH = "/Users/hongbikim/Dev/dacon_finance/" + file
    file_id = uuid.uuid4().hex

    # PyMuPDF 로더 인스턴스 생성
    loader = PyMuPDFLoader(FILE_PATH)

    # 문서 로드
    docs = loader.load()

    for doc in docs:
        # 페이지별로 처리
        page_num = doc.metadata.get('page', 0)
        content = clean_text(doc.page_content)
        
        # 조문/장절 정보 추출
        legal_structure = extract_legal_structure(content)
        
        # 텍스트 분할
        splits = base_splitter.split_text(content)
        
        # 임시로 분할된 청크들 생성
        temp_splits = []
        for i, split_content in enumerate(splits):
            chunk_structure = extract_legal_structure(split_content)
            
            split_doc = Document(
                page_content=split_content,
                metadata={
                    # **doc.metadata,
                    # 'chunk_index': i,
                    'file_name': FILE_PATH.split("/")[-1],
                    'file_id': file_id,
                    'page_chunk_id': f"page_{page_num}_chunk_{i}",
                    # 'legal_article': chunk_structure.get('article'),
                    # 'legal_chapter': chunk_structure.get('chapter'),
                    # 'legal_section': chunk_structure.get('section'),
                    # 'legal_paragraph': chunk_structure.get('paragraph'),
                }
            )
            temp_splits.append(split_doc)
        
        # 짧은 청크들 합치기
        merged_splits = _merge_short_chunks(temp_splits, page_num)
        try:
            print(f"Completed")
        except Exception as e:
            print(f"Error adding documents for file {i}th {file}: {e}")
            error_files.append(file)
            continue
        all_splits.extend(merged_splits)

Completed
Completed
Completed
Completed
Completed
Completed
Completed
Completed
Completed
Completed
Completed
Completed
Completed
Completed
Completed
Completed
Completed
Completed
Completed
Completed
Completed
Completed
Completed
Completed
Completed
Completed
Completed
Completed
Completed
Completed
Completed
Completed
Completed
Completed
Completed
Completed
Completed
Completed
Completed
Completed
Completed
Completed
Completed
Completed
Completed
Completed
Completed
Completed


In [74]:
all_splits

[Document(metadata={'file_name': 'rfc4408.pdf', 'file_id': 'ff338207e6714edfbbd66070f9b5dccd', 'page_chunk_id': 'page_0_chunk_0', 'chunk_length': 486}, page_content='Network Working Group M. Wong\nRequest for Comments: 4408 W. Schlitt\nCategory: Experimental April 2006 Sender Policy Framework (SPF) for Authorizing Use of Domains in E-Mail, Version 1\nStatus of This Memo This memo defines an Experimental Protocol for the Internet community. It does not specify an Internet standard of any kind. Discussion and suggestions for improvement are requested. Distribution of this memo is unlimited.\nCopyright Notice Copyright (C) The Internet Society (2006).'),
 Document(metadata={'file_name': 'rfc4408.pdf', 'file_id': 'ff338207e6714edfbbd66070f9b5dccd', 'page_chunk_id': 'page_0_chunk_1', 'chunk_length': 506}, page_content='IESG Note The following documents (RFC 4405, RFC 4406, RFC 4407, and RFC 4408) are published simultaneously as Experimental RFCs, although there is no general technical conse

In [75]:
print(len(files))
print(len(all_splits))

107
296


In [29]:
print(len(all_splits))

1098


In [76]:
from langchain_huggingface.embeddings import HuggingFaceEndpointEmbeddings

model_name = "nlpai-lab/KURE-v1"

hf_embeddings = HuggingFaceEndpointEmbeddings(
    model=model_name,
    task="feature-extraction",
    huggingfacehub_api_token=os.environ["HUGGINGFACEHUB_API_TOKEN"],
)

In [18]:
# Qdrant 클라이언트 생성 (메모리 기반)
# ":memory:" 옵션을 사용하면 휘발성(In-Memory) 데이터베이스로 실행됨.
# client = QdrantClient(":memory:")

client = QdrantClient(
    host="localhost",
    port=6333,
    # path = "/Users/hongbikim/Dev/dacon_finance"
)

# # 1. 먼저 컬렉션 생성 (임베딩 차원에 맞춰서)
# collection_name = "rag-finance"

# # 임베딩 차원 확인 (hf_embeddings의 차원)
# sample_embedding = hf_embeddings.embed_query("test")
# embedding_dim = len(sample_embedding)
# print(f"임베딩 차원: {embedding_dim}")

# # 컬렉션 생성
# client.create_collection(
#     collection_name=collection_name,
#     vectors_config=VectorParams(
#         size=embedding_dim,  # 임베딩 차원
#         distance=Distance.COSINE  # 거리 측정 방식
#     )
# )

# # Qdrant 벡터 저장소 객체 생성
# vector_store = QdrantVectorStore(
#     client = client,
#     collection_name = "rag-finance",
#     embedding = hf_embeddings
# )

In [12]:
cloud_client = QdrantClient(
    url=QDRANT_URL, 
    api_key=QDRANT_api_key,
)

In [13]:
from qdrant_client.models import PointStruct

def migrate_collection(local_client, cloud_client, collection_name):
    """로컬에서 클라우드로 컬렉션 마이그레이션"""
    
    print("1. 로컬 컬렉션 정보 가져오기...")
    collection_info = local_client.get_collection(collection_name)
    
    print("2. 클라우드에 컬렉션 생성...")
    try:
        cloud_client.create_collection(
            collection_name=collection_name,
            vectors_config=collection_info.config.params.vectors
        )
        print(f"✅ 컬렉션 '{collection_name}' 생성 완료")
    except Exception as e:
        print(f"⚠️ 컬렉션 생성 오류 (이미 존재할 수 있음): {e}")
    
    print("3. 데이터 이전 시작...")
    
    # 배치로 나눠서 이전
    batch_size = 100
    offset = None
    total_migrated = 0
    
    while True:
        # 로컬에서 데이터 가져오기
        result = local_client.scroll(
            collection_name=collection_name,
            limit=batch_size,
            offset=offset,
            with_payload=True,
            with_vectors=True
        )
        
        points, next_offset = result
        
        if not points:
            break
        
        # PointStruct 형식으로 변환
        converted_points = []
        for point in points:
            converted_point = PointStruct(
                id=point.id,
                vector=point.vector,
                payload=point.payload
            )
            converted_points.append(converted_point)
        
        # 클라우드에 업로드
        try:
            cloud_client.upsert(
                collection_name=collection_name,
                points=converted_points
            )
            total_migrated += len(converted_points)
            print(f"✅ {total_migrated}개 포인트 이전 완료...")
            
        except Exception as e:
            print(f"❌ 배치 업로드 실패: {e}")
            return False
        
        # 다음 배치로
        offset = next_offset
        if offset is None:
            break
    
    print(f"🎉 마이그레이션 완료! 총 {total_migrated}개 포인트 이전됨")
    return True

# 사용법
local_client = QdrantClient(host="localhost", port=6333)
cloud_client = QdrantClient(
    url=QDRANT_URL, 
    api_key=QDRANT_api_key,
)

# 마이그레이션 실행
success = migrate_collection(local_client, cloud_client, "rag-finance")

if success:
    # 검증
    local_count = local_client.get_collection("rag-finance").points_count
    cloud_count = cloud_client.get_collection("rag-finance").points_count
    print(f"검증 - 로컬: {local_count}개, 클라우드: {cloud_count}개")

1. 로컬 컬렉션 정보 가져오기...
2. 클라우드에 컬렉션 생성...
⚠️ 컬렉션 생성 오류 (이미 존재할 수 있음): Unexpected Response: 409 (Conflict)
Raw response content:
b'{"status":{"error":"Wrong input: Collection `rag-finance` already exists!"},"time":0.022527237}'
3. 데이터 이전 시작...
✅ 100개 포인트 이전 완료...
✅ 200개 포인트 이전 완료...
✅ 300개 포인트 이전 완료...
✅ 400개 포인트 이전 완료...
✅ 500개 포인트 이전 완료...
✅ 600개 포인트 이전 완료...
✅ 700개 포인트 이전 완료...
✅ 800개 포인트 이전 완료...
✅ 900개 포인트 이전 완료...
✅ 1000개 포인트 이전 완료...
✅ 1100개 포인트 이전 완료...
✅ 1200개 포인트 이전 완료...
✅ 1300개 포인트 이전 완료...
✅ 1400개 포인트 이전 완료...
✅ 1500개 포인트 이전 완료...
✅ 1600개 포인트 이전 완료...
✅ 1700개 포인트 이전 완료...
✅ 1800개 포인트 이전 완료...
✅ 1900개 포인트 이전 완료...
✅ 2000개 포인트 이전 완료...
✅ 2100개 포인트 이전 완료...
✅ 2200개 포인트 이전 완료...
✅ 2300개 포인트 이전 완료...
✅ 2400개 포인트 이전 완료...
✅ 2500개 포인트 이전 완료...
✅ 2600개 포인트 이전 완료...
✅ 2700개 포인트 이전 완료...
✅ 2800개 포인트 이전 완료...
✅ 2900개 포인트 이전 완료...
✅ 3000개 포인트 이전 완료...
✅ 3100개 포인트 이전 완료...
✅ 3200개 포인트 이전 완료...
✅ 3300개 포인트 이전 완료...
✅ 3400개 포인트 이전 완료...
✅ 3500개 포인트 이전 완료...
✅ 3600개 포인트 이전 완료...
✅ 3700개 포인트 

In [14]:
# 현재 points 구조 확인
points = local_client.scroll(
    collection_name="rag-finance",
    limit=1,
    with_payload=True,
    with_vectors=True
)

if points[0]:
    sample_point = points[0][0]
    print(f"Point 타입: {type(sample_point)}")
    print(f"Point ID: {sample_point.id}")
    print(f"Vector 타입: {type(sample_point.vector)}")
    print(f"Payload 키들: {list(sample_point.payload.keys()) if sample_point.payload else 'None'}")

Point 타입: <class 'qdrant_client.http.models.models.Record'>
Point ID: 000cfefe-41f5-45e5-b36a-6d3b475be14f
Vector 타입: <class 'list'>
Payload 키들: ['page_content', 'metadata']


In [4]:
local_client = QdrantClient(host="localhost", port=6333)

In [21]:
def reset_collection(collection_name="rag-finance"):
    """컬렉션 초기화"""
    try:
        # 기존 컬렉션 삭제
        client.delete_collection(collection_name)
        print(f"✅ 컬렉션 '{collection_name}' 삭제 완료")
    except Exception as e:
        print(f"⚠️  컬렉션 삭제 중 오류: {e}")
    
    # 임베딩 차원 확인
    sample_embedding = hf_embeddings.embed_query("test")
    embedding_dim = len(sample_embedding)
    
    # 새 컬렉션 생성
    client.create_collection(
        collection_name=collection_name,
        vectors_config=VectorParams(
            size=embedding_dim,
            distance=Distance.COSINE
        )
    )
    print(f"✅ 새 컬렉션 '{collection_name}' 생성 완료")
    
    # 새 벡터스토어 반환
    return QdrantVectorStore(
        client=client,
        collection_name=collection_name,
        embedding=hf_embeddings
    )

# # 사용법
vector_store = reset_collection()

✅ 컬렉션 'rag-finance' 삭제 완료
✅ 새 컬렉션 'rag-finance' 생성 완료


In [126]:
vector_store.add_documents(documents=all_splits)

['0ab8dedf6933470db3c6fb746f416603',
 '81320a5b2df241c197d3b59524b4b264',
 'e1c96f2024bc4e12ada58fa4639b7670',
 '915fc725007c4bb6bc86018e32913cf5',
 'c72575ad9c6b40e79761cc68a8eea202',
 '23ef8323bb0d43a488ea4b16c20e3f73',
 '0b3ce69c416d4abda7d3bab6d7387fbf',
 'a363f2b8fc574c258bd848f8339fe7fe',
 'a05933c5e2944a179153960adc428a89',
 'b99584526f904a63910d845599ebf565',
 '058e3de425d848b0a70429e1d73e330e',
 '4cdaf4020087426fab860e22b3581017',
 'bd04aa4e48644a97a91afff46eeaae9d',
 '755516c2aa394da7b2262adc51d94cf6',
 '78a967bab3804014ba6a44b315f2ec17',
 '45537288ac0e412b87297873b225505d',
 'dafff347cdb74bcb8860581fdb7d2481',
 '44b7984d7182485682e53d27e4241c49',
 '823e3c2860714eb783aeecf71521b11a',
 'ab765d285b424329b1ec78f8063bfacd',
 '2e6a9454cd1c4149b2292ae37f9c3d6a',
 '7e2430a94e8b42fb81384bfbdf1e83a5',
 'f7a20aa776b94c06bb3ee4bc3bcadfa9',
 'd1a197c6f6b441738aee235e005ef6a7',
 '8c440f4bdda64cd29712cdb39241a471',
 '1ac862c1179546f1a6b2313764f3f5fb',
 '28c8dba5eb274655bdbf2330bb2af8fb',
 

In [77]:
client = QdrantClient(
    url=os.environ["QDRANT_URL"],
    api_key=os.environ["QDRANT_api_key"],
    # port=6333,
    # path = "/Users/hongbikim/Dev/dacon_finance"
)

collections = client.get_collections()
collections

CollectionsResponse(collections=[CollectionDescription(name='rag-finance')])

In [78]:
vector_store = QdrantVectorStore(
    client = client,
    collection_name = "rag-finance",
    embedding = hf_embeddings
)

In [79]:
vector_store.add_documents(documents=all_splits)

['bca9ecf2262243ad90aed6806cc8ffbb',
 '2bb3ca4b112442298afb8adae0845526',
 'c09f8135038f4f7085e520f0835f15dc',
 'c8f41f7c3581430ea71bbc600ef49de8',
 'ef3c48f9b09e4329880d5fdebbdf10a2',
 '9ddfa06322c944f697457c2e1ee09cd9',
 'f1764c4feb934c9397ad4b32f165e6c8',
 '4e30df3e666746bba97511e7d8e73efe',
 '68dd186bc76e4a89a920a1b7b82f0f1d',
 '6abaedbeb66a48d58fcc050ba6d79915',
 '3d9a6c057f164239aab8ffca229eae23',
 '9fe9c8b76596429a9e024828c54fb33a',
 'f7aa588f1c514a4587ea45994cf87238',
 '394f3495e6544963a0ffc58153660057',
 '37e0ce34355c4331ab921fa4f158c62d',
 '9dfeec9b683e46aaa888b95f65c2c254',
 '8abfb335dd8741bf99d223f53fa8be7d',
 'e470739e9efe4318ac68a621b2c1f0ec',
 '52e19f60f88b43ceaa89d9e093d875e9',
 '303d3a2c63e8470587f9b1a87b0540aa',
 '022ef06772a0465a90bc893c2f2fa6ef',
 'aab9e31ce4b04a649ba2b444ceff2fda',
 '15be79f6cc0846a89e88fbe0cf83e9c0',
 '92b9bcd449f54fbb86e7304355e14b1a',
 'f6265a50c8694f6eb503ed70567f4597',
 '811ceea2ef12452e9ae749390b2c2e8a',
 'be9ce7a6f8f04b9aa964d6b732f4d4fd',
 

In [60]:
collection_info = client.get_collection("rag-finance")
print(len(all_splits))
print(f"포인터 개수: {collection_info.points_count}")
print(f"컬렉션 상태: {collection_info.status}")


69
포인터 개수: 11675
컬렉션 상태: green


In [45]:
11606 - 10508

1098

In [44]:
collection_info = client.get_collection("rag-finance")
print(len(all_splits))
print(f"포인터 개수: {collection_info.points_count}")
print(f"컬렉션 상태: {collection_info.status}")

1098
포인터 개수: 11606
컬렉션 상태: green


In [27]:
test = pd.read_csv("/Users/hongbikim/Dev/dacon_finance/open/test.csv")

In [ ]:
q_i = 0
query = test['Question'][q_i].split("\n")[0]
# print(test['Question'][q_i])
# print(query)

results = vector_store.similarity_search(
    query = query,
    k = 10,
    # filter = models.Filter(must=[models.FieldCondition(
    #     key = "metadata.page", # 특정 필드(예: page) 기반 필터링
    #     match = models.MatchValue(value=2), # page 1인 문서만 검색
    # )])
)

ctxs = []
for result in results:
    # print(result.metadata['file_name'])
    ctxs.append(result.page_content)
ctxs

In [32]:
test['Question'][q_i]

'금융산업의 이해와 관련하여 금융투자업의 구분에 해당하지 않는 것은?\n1 소비자금융업\n2 투자자문업\n3 투자매매업\n4 투자중개업\n5 보험중개업'

In [ ]:
result.metadata['file_name']

In [37]:
ref = ""
for c_i, ctx in enumerate(ctxs):
    ref += str(c_i+1) + "번째 chunk\n" + ctx + "\n\n"
print(ref)

1번째 chunk: 법제처 20 국가법령정보센터
자본시장과 금융투자업에 관한 법률 시행령
1. 별표 1에 따른 금융투자업의 종류 중 다음 각 목의 금융투자업의 경우: 해당 금융투자업에 속하는 금융투자상품
중 증권, 장내파생상품, 장외파생상품 각각을 기준으로 하여 인가받지 않은 다른 업무 단위
가. 투자매매업
나. 투자매매업(인수업은 제외한다)
다. 투자매매업(인수업만 해당한다)
2. 별표 1에 따른 금융투자업의 종류 중 투자중개업(같은 표에 따라 인가업무 단위가 2l-1-1, 2l-1-2, 2-14-1 및 2-
14-2인 투자중개업은 제외한다)의 경우: 투자중개업에 속하는 인가업무 단위 중 인가받지 않은 다른 업무 단위

2번째 chunk: 1. 금융투자업의 종류(투자매매업, 투자중개업, 집합투자업 및 신탁업을 말하되, 투자매매업 중 인수업을 포함한다)
2. 금융투자상품(집합투자업의 경우에는 제229조에 따른 집합투자기구의 종류를 말하며, 신탁업의 경우에는 제
103조제1항 각 호의 신탁재산을 말한다)의 범위(증권, 장내파생상품 및 장외파생상품을 말하되, 증권 중 국채증권
, 사채권, 그 밖에 대통령령으로 정하는 것을 포함하고 파생상품 중 주권을 기초자산으로 하는 파생상품ㆍ그 밖에
대통령령으로 정하는 것을 포함한다)
3. 투자자의 유형(전문투자자 및 일반투자자를 말한다. 이하 같다)

3번째 chunk: 법제처 9 국가법령정보센터
자본시장과 금융투자업에 관한 법률 시행령
무와 관련된 분석정보 등을 제공하는 경우
10. 다른 법령에 따라 건축물 및 주택의 임대관리 등 부동산의 관리대행, 부동산의 이용ㆍ개발 및 거래에 대한 상담,
그 밖에 부동산의 투자ㆍ운용에 관한 자문 등의 업무를 영위하는 경우

⑤ 법 제7조제6항에 따라 다음 각 호의 어느 하나에 해당하는 경우에는 해당 호의 금융투자업으로 보지 아니한다.

1. 법 제7조제6항제1호의 경우: 투자중개업
2. 법 제7조제6항제2호의 경우: 투자매매업
3. 법 제7조제6항제3호의 경우: 투자매매업 또는 투자중개업